In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('train.csv')
y = train_df['target']
train_df = train_df.drop(['target'], axis = 1)
test_df = pd.read_csv('test.csv')
df = pd.concat([train_df, test_df])